# Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas

from path import Path
import plotly.express as px

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path,index_col = 0)
df_crypto.head()

In [ ]:
# Keep all the cryptocurrencies that are being traded.
df_crypto = df_crypto[df_crypto.IsTrading.eq(True)]
df_crypto.shape

In [ ]:
# Keep all the cryptocurrencies that have a working algorithm.
df_crypto = df_crypto[df_crypto.IsTrading.eq(True)]
df_crypto.shape

In [ ]:
# Remove the "IsTrading" column. 
df_crypto = df_crypto.drop(["IsTrading"],axis = 1)
df_crypto

In [ ]:
# Remove rows that have at least 1 null value.
df_crypto = df_crypto.dropna()
df_crypto

In [ ]:
# Keep the rows where coins are mined.
df_crypto = df_crypto.loc[df_crypto['TotalCoinsMined'] > 0]
df_crypto.head()

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names = df_crypto.filter(['CoinName'], axis=1)
names

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df_crypto = df_crypto.drop(['CoinName'],axis = 1)
df_crypto

In [ ]:
# Use get_dummies() to create variables for text features.
crypto = pd.get_dummies(df_crypto['Algorithm'])
dummy = pd.get_dummies(df_crypto['ProofType'])
combined = pd.concat([crypto,dummy],axis =1)
df = df_crypto.merge(combined,left_index = True,right_index = True)
df = df.drop(['Algorithm','ProofType'],axis = 1)
df

In [ ]:
# Standardize the data with StandardScaler().
df_scaled = StandardScaler().fit_transform(df)
print(df_scaled)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca

In [ ]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = df_pca, columns = ['PC1','PC2','PC3'], index = df_crypto.index
)
pcs_df

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df_crypto, pcs_df], axis =1)


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = names['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"]= model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True, selectable=True)

In [ ]:
# Print the total number of tradable cryptocurrencies.
print(clustered_df.count)

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled =MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
print(scaled)

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_df = pd.DataFrame(
    data = scaled,columns = ["TotalCoinSupply","TotalCoinsMined"], index = clustered_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_df = pd.concat([new_df,clustered_df['CoinName']],axis =1)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.concat([new_df,clustered_df["Class"]],axis =1)

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class",hover_cols = ["CoinName"])